In [ ]:
!pip install datasets transformers sentencepiece accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from google.colab import drive
from pathlib import Path
import os
from datasets import load_dataset, load_metric, Dataset, Split
import datasets
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorWithPadding, pipeline
import pickle
import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, get_linear_schedule_with_warmup, T5ForConditionalGeneration, T5Tokenizer
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from datasets import load_metric
import numpy as np
import random

In [ ]:
# Set the random seeds for reproducibility
torch.manual_seed(25)
np.random.seed(25)
random.seed(25)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving scripts.csv to scripts.csv


In [ ]:
df = pd.read_csv("scripts.csv").drop(["Description", "Unnamed: 0"], axis = 1)
df = df.dropna()
mapping = {'I': 0, 'E': 1}
df['Introversion/Extroversion'] = df['Introversion/Extroversion'].map(mapping)
mapping = {"N": 0, "S": 1}
df["Sensing/Intuition"] = df["Sensing/Intuition"].map(mapping)
mapping = {"T": 0, "F": 1}
df["Thinking/Feeling"] = df["Thinking/Feeling"].map(mapping)
mapping = {"J": 0, "P": 1}
df["Judging/Perceiving"] = df["Judging/Perceiving"].map(mapping)
df = df.reset_index(drop=True)

In [ ]:
def create_ds(df, field):
  df_temp = df[[field, "Dialogue"]]
  df_temp = df_temp.rename(columns={field: "labels", "Dialogue": "input"})
  dataset = Dataset.from_pandas(df_temp)
  return dataset

In [ ]:
IE_ds = create_ds(df, 'Introversion/Extroversion')
SI_ds = create_ds(df, 'Sensing/Intuition')
TF_ds = create_ds(df, 'Thinking/Feeling')
JP_ds = create_ds(df, 'Judging/Perceiving')

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
max_input_length = 512
            
# Tokenize the data and convert to Hugging Face Datasets
def tokenize_function(examples):
    return tokenizer(examples['input'], padding='max_length', max_length=max_input_length, truncation=True)

tokenized_IE_ds = IE_ds.map(tokenize_function, batched = True)
tokenized_SI_ds = SI_ds.map(tokenize_function, batched = True)
tokenized_TF_ds = TF_ds.map(tokenize_function, batched = True)
tokenized_JP_ds = JP_ds.map(tokenize_function, batched = True)
split_IE_ds = tokenized_IE_ds.train_test_split(test_size=0.2)
split_SI_ds = tokenized_SI_ds.train_test_split(test_size=0.2)
split_TF_ds = tokenized_TF_ds.train_test_split(test_size=0.2)
split_JP_ds = tokenized_JP_ds.train_test_split(test_size=0.2)

Map:   0%|          | 0/532 [00:00<?, ? examples/s]

Map:   0%|          | 0/532 [00:00<?, ? examples/s]

Map:   0%|          | 0/532 [00:00<?, ? examples/s]

Map:   0%|          | 0/532 [00:00<?, ? examples/s]

In [ ]:
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = torch.nn.CrossEntropyLoss()(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    learning_rate=3e-6,
    logging_strategy = "epoch",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True
)

# Define the Trainer
trainer_IE_ds = MyTrainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).cuda(),
    args=training_args,
    train_dataset=split_IE_ds['train'],
    eval_dataset=split_IE_ds['test'],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer_IE_ds.train()

evaluation = trainer_IE_ds.evaluate(eval_dataset=split_IE_ds['test'])
print(evaluation)
evaluation = trainer_IE_ds.evaluate(eval_dataset=split_IE_ds['train'])
print(evaluation)

<ipython-input-12-99620c2aa25f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695000,0.690755,0.523364
2,0.685100,0.690497,0.514019
3,0.680500,0.682875,0.495327
4,0.654600,0.683715,0.532710
5,0.630800,0.680620,0.523364
6,0.585200,0.691415,0.551402
7,0.558700,0.699763,0.551402
8,0.512400,0.717443,0.532710
9,0.474500,0.729862,0.542056
10,0.434200,0.743517,0.542056


{'eval_loss': 0.6806203722953796, 'eval_accuracy': 0.5233644859813084, 'eval_runtime': 1.9755, 'eval_samples_per_second': 54.164, 'eval_steps_per_second': 13.667, 'epoch': 15.0}
{'eval_loss': 0.583866536617279, 'eval_accuracy': 0.7552941176470588, 'eval_runtime': 7.7739, 'eval_samples_per_second': 54.67, 'eval_steps_per_second': 13.764, 'epoch': 15.0}


In [ ]:
def get_n_confident(ds, trainer):
  preds = trainer.predict(ds['test'])
  probs = preds[0][:, 1]

  # Get the predicted labels and ground truth labels
  predicted_labels = preds[1]
  true_labels = [example['labels'] for example in ds['test']]

  softmax_probabilities = np.exp(preds[0]) / np.sum(np.exp(preds[0]), axis=1, keepdims=True)

  one_probabilities = softmax_probabilities[:, 1]

  # Create a list of tuples containing the index of each instance and its softmax probability
  one_prediction_tuples = [(i, prob) for i, prob in enumerate(one_probabilities)]

  # Sort the list of tuples by softmax probability
  oneprediction_tuples_sorted = sorted(one_prediction_tuples, key=lambda x: x[1], reverse=True)[:10]

  zero_probabilities = softmax_probabilities[:, 0]

  # Create a list of tuples containing the index of each instance and its softmax probability
  zero_prediction_tuples = [(i, prob) for i, prob in enumerate(zero_probabilities)]

  # Sort the list of tuples by softmax probability
  zero_prediction_tuples_sorted = sorted(zero_prediction_tuples, key=lambda x: x[1], reverse=True)[:10]

  rows = []
  for idx, confidence in zero_prediction_tuples_sorted:
      row = {'input': ds['test'][idx]['input'], 'confidence': confidence, "actual": ds['test'][idx]['labels'], "predicted": 0}
      rows.append(row)

  for idx, confidence in oneprediction_tuples_sorted:
      row = {'input': ds['test'][idx]['input'], 'confidence': confidence, "actual": ds['test'][idx]['labels'], "predicted": 1}
      rows.append(row)

  # Displays the top 20 confidences of correctly-evaluated inputs as a dataframe.    
  df = pd.DataFrame(rows)

  # Compare predicted and actual labels and count the number of correct predictions
  correct_predictions = df['actual'] == df['predicted']
  n_correct = sum(correct_predictions)

  # Compute the accuracy as the ratio of correct predictions to total instances
  accuracy = n_correct / len(df)

  print(f"Accuracy = {accuracy:.2f}")
  return df

In [ ]:
get_n_confident(split_IE_ds, trainer_IE_ds)

Accuracy = 0.75


,input,confidence,actual,predicted
0,JEROME emerges from the incinerator room into ...,0.728550,0,0
1,We're going to freeze to death.\nMaybe somebod...,0.721187,0,0
2,Where's the coffee.\nDALLAS looks at his grogg...,0.714467,0,0
3,ALICE comes into the MS. She is carrying a sho...,0.714444,0,0
4,Elena. My name's Elena.\nELENA. He nods his he...,0.699102,0,0
5,"...Hello.\n...What? Who is this?\nYeah, I'm st...",0.689167,0,0
6,RUDY BUTLER -- hard eyes and an easy smile; ob...,0.682013,1,0
7,"You're Julia, right?\nI'm brother Frank.\nThat...",0.668568,1,0
8,"TINA rushes away, hands over her ears.\nTINA o...",0.660991,1,0
9,She must have done something with the child. W...,0.656078,0,0


In [ ]:
trainer_SI_ds = MyTrainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).cuda(),
    args=training_args,
    train_dataset=split_SI_ds['train'],
    eval_dataset=split_SI_ds['test'],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer_SI_ds.train()

evaluation = trainer_SI_ds.evaluate(eval_dataset=split_SI_ds['test'])
print(evaluation)
evaluation = trainer_SI_ds.evaluate(eval_dataset=split_SI_ds['train'])
print(evaluation)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch,Training Loss,Validation Loss,Accuracy
1,0.693400,0.681800,0.570093
2,0.690800,0.681044,0.588785
3,0.681100,0.675601,0.588785
4,0.666100,0.663637,0.598131
5,0.640700,0.654597,0.607477
6,0.616800,0.646114,0.616822
7,0.579900,0.636488,0.644860
8,0.544600,0.626151,0.663551
9,0.492600,0.623780,0.663551
10,0.457300,0.622746,0.663551


{'eval_loss': 0.6227462887763977, 'eval_accuracy': 0.6635514018691588, 'eval_runtime': 1.9773, 'eval_samples_per_second': 54.114, 'eval_steps_per_second': 13.655, 'epoch': 15.0}
{'eval_loss': 0.402529239654541, 'eval_accuracy': 0.8752941176470588, 'eval_runtime': 7.7535, 'eval_samples_per_second': 54.814, 'eval_steps_per_second': 13.8, 'epoch': 15.0}


In [ ]:
get_n_confident(split_SI_ds, trainer_SI_ds)

Accuracy = 0.75


,input,confidence,actual,predicted
0,He's been tested for drugs?\nRAILLY'S POV THRO...,0.828133,0,0
1,"EMMA smiles. Alexander stands, just watching h...",0.791559,0,0
2,"FINNEGAN, bathed in the last light of day, all...",0.764962,0,0
3,Back! Don't go near it. We don't know where it...,0.760359,1,0
4,ELLIE Dr. Grant! Dr. Grant! Grant looks up.\nS...,0.759556,0,0
5,He sounds like a real gentleman.\nI'm so sorry...,0.759276,1,0
6,As you wish. Westley is perhaps half a dozen y...,0.741497,0,0
7,"OSBORN, fiftyish. Harry has already inherited ...",0.734334,1,0
8,"LESTAT, a hooded figure in the corner, smiles ...",0.723847,1,0
9,Stop it. Stop it! They roll on the floor. He t...,0.720875,1,0


In [ ]:
trainer_TF_ds = MyTrainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).cuda(),
    args=training_args,
    train_dataset=split_TF_ds['train'],
    eval_dataset=split_TF_ds['test'],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer_TF_ds.train()

evaluation = trainer_TF_ds.evaluate(eval_dataset=split_TF_ds['test'])
print(evaluation)
evaluation = trainer_TF_ds.evaluate(eval_dataset=split_TF_ds['train'])
print(evaluation)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch,Training Loss,Validation Loss,Accuracy
1,0.694400,0.688183,0.551402
2,0.683700,0.677996,0.607477
3,0.665100,0.659670,0.682243
4,0.634200,0.636424,0.682243
5,0.590300,0.634175,0.635514
6,0.551300,0.598205,0.700935
7,0.531100,0.617901,0.663551
8,0.498100,0.593682,0.682243
9,0.470100,0.602385,0.682243
10,0.430600,0.592294,0.663551


{'eval_loss': 0.5922939777374268, 'eval_accuracy': 0.6635514018691588, 'eval_runtime': 1.9943, 'eval_samples_per_second': 53.654, 'eval_steps_per_second': 13.539, 'epoch': 15.0}
{'eval_loss': 0.386423259973526, 'eval_accuracy': 0.8635294117647059, 'eval_runtime': 7.8229, 'eval_samples_per_second': 54.328, 'eval_steps_per_second': 13.678, 'epoch': 15.0}


In [ ]:
get_n_confident(split_TF_ds, trainer_TF_ds)

Accuracy = 0.85


,input,confidence,actual,predicted
0,Where's the coffee.\nDALLAS looks at his grogg...,0.909977,0,0
1,Time for another one.\nThe small matter of a s...,0.894435,0,0
2,"THIBADEAUX, a Cajun who carries himself with a...",0.875882,0,0
3,"Turn the goddamn thing off, will ya.\nGlad to ...",0.869470,0,0
4,MAX Just overnight is all. Tomorrow I'll\nMAX ...,0.868290,0,0
5,DOYLE is at the wheel. BLACK PUSHER is sitting...,0.867692,0,0
6,"Second Foot, wheel right, advance quick step.....",0.863313,0,0
7,"I'm busy, Floyd.\nWhich one?\nYou sure he didn...",0.861696,0,0
8,How much you gonna pay me? Huh? I'd be doing y...,0.843089,0,0
9,How much you gonna pay me? Huh? I'd be doing y...,0.843089,0,0


In [ ]:
trainer_JP_ds = MyTrainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).cuda(),
    args=training_args,
    train_dataset=split_JP_ds['train'],
    eval_dataset=split_JP_ds['test'],
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer_JP_ds.train()

evaluation = trainer_JP_ds.evaluate(eval_dataset=split_JP_ds['test'])
print(evaluation)
evaluation = trainer_JP_ds.evaluate(eval_dataset=split_JP_ds['train'])
print(evaluation)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch,Training Loss,Validation Loss,Accuracy
1,0.691000,0.682012,0.570093
2,0.684400,0.680317,0.570093
3,0.674200,0.678002,0.570093
4,0.665000,0.677305,0.579439
5,0.652000,0.670128,0.579439
6,0.626500,0.668105,0.588785
7,0.587500,0.667427,0.588785
8,0.550100,0.668186,0.598131
9,0.519900,0.677098,0.560748
10,0.490500,0.678417,0.588785


{'eval_loss': 0.6674271821975708, 'eval_accuracy': 0.5887850467289719, 'eval_runtime': 1.998, 'eval_samples_per_second': 53.554, 'eval_steps_per_second': 13.514, 'epoch': 15.0}
{'eval_loss': 0.5327287912368774, 'eval_accuracy': 0.8047058823529412, 'eval_runtime': 7.7861, 'eval_samples_per_second': 54.584, 'eval_steps_per_second': 13.742, 'epoch': 15.0}


In [ ]:
get_n_confident(split_JP_ds, trainer_JP_ds)

Accuracy = 0.70


,input,confidence,actual,predicted
0,STARGHER'S HANDS COME AT HER FROM BEHIND! One ...,0.678755,1,0
1,"Hello, Mr. Deckard. My name is Rachael. Deckar...",0.675241,0,0
2,"You're Julia, right?\nI'm brother Frank.\nThat...",0.656577,1,0
3,"You're Julia, right?\nI'm brother Frank.\nThat...",0.650883,1,0
4,We're going to freeze to death.\nMaybe somebod...,0.647863,0,0
5,"LARRY COTTON, the other his wife JULIA. Clearl...",0.646232,0,0
6,"Yes, with Fedens and the children. He wants so...",0.636352,0,0
7,Geoffrey gave me your monograph\nLove? Romanti...,0.617718,0,0
8,"LYDIA - a dowdy, waif-like sparrow of a thing,...",0.604920,1,0
9,That is of no interest. And now he turns and l...,0.590865,0,0
